In [1]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2018,23)

In [2]:
input_text = """pos=<0,0,0>, r=4
pos=<1,0,0>, r=1
pos=<4,0,0>, r=3
pos=<0,2,0>, r=1
pos=<0,5,0>, r=3
pos=<0,0,3>, r=1
pos=<1,1,1>, r=1
pos=<1,1,2>, r=1
pos=<1,3,1>, r=1"""
input_lines = input_text.splitlines()


In [3]:
import re
pattern_input = re.compile(r'pos=<(-?\d+),(-?\d+),(-?\d+)>, r=(\d+)')
def parse_input(lines):
    bots = {}
    for line in lines:
        match = pattern_input.findall(line)
        if match:
            pos_x, pos_y, pos_z, radius = [int(num) for num in match[0]]
            bots[(pos_x, pos_y, pos_z)] = radius
    return bots


In [4]:
from grid import manhattan_distance
nanobots = parse_input(input_lines)
max_signal = min([signal for signal in nanobots.values()])
max_nanobot = None
for nanobot, signal in nanobots.items():
    if signal == max_signal:
        max_nanobot = nanobot
print(max_nanobot, max_signal)
in_range = []
for nanobot in nanobots:
    if manhattan_distance(max_nanobot, nanobot) <= max_signal:
        in_range.append(nanobot)

print(len(in_range))
    
print(manhattan_distance((0,0,0),(0,1000,0)))

(1, 3, 1) 1
1
1000


In [5]:
input_text = """pos=<10,12,12>, r=2
pos=<12,14,12>, r=2
pos=<16,12,12>, r=4
pos=<14,14,14>, r=6
pos=<50,50,50>, r=200
pos=<10,10,10>, r=5"""
input_lines = input_text.splitlines()
nanobots  = parse_input(input_lines)

In [6]:
import numpy as np
from scipy.spatial import KDTree

# Manhattan distance calculation helper function

def manhattan_distance2(p1, p2):
    return np.sum(np.abs(np.array(p1) - np.array(p2)))

def get_points_in_range(tree, points):
    all_points_in_range = set()
    
    for point, range_val in points.items():
        # Query the tree for all points within Chebyshev (L_infinity) distance.
        # We first find points in a large enough box.
        nearby_indices = tree.query_ball_point(point, r=range_val, p=1)  # `p=1` is for Manhattan distance
        for idx in nearby_indices:
            nearby_point = tuple(point_coords[idx])
            if manhattan_distance2(point, nearby_point) <= range_val:
                all_points_in_range.add(nearby_point)
    
    return all_points_in_range


point_coords = np.array(list(nanobots.keys()))
tree = KDTree(point_coords)
in_range_points = get_points_in_range(tree, nanobots)
print(in_range_points)

{(14, 14, 14), (12, 14, 12), (10, 10, 10), (50, 50, 50), (10, 12, 12), (16, 12, 12)}


In [7]:
import itertools
def find_all_points(points):
    # get max values
    max_x = max([point[0] for point in points])
    max_y = max([point[1] for point in points])
    max_z = max([point[2] for point in points])
    min_x = min([point[0] for point in points])
    min_y = min([point[1] for point in points])
    min_z = min([point[2] for point in points])
    points = set()
    # walk row/col combinations
    # I need to remember this, it is faster than for x in range, for y in range
    # also keeping current_point in a tuple, seemed to be faster than (col, row)
    for current_point in itertools.product(range(min_x, max_x + 1), range(min_y, max_y + 1), range(min_z, max_z + 1)):
        points.add(current_point)
    return points


my_points = set()
my_points.update(find_all_points(nanobots))
print(len(my_points))
counts = {}
for point in my_points:
    counts[point] = 0
    for nanobot, signal_range in nanobots.items():
        if manhattan_distance(point, nanobot) <= signal_range:
            counts[point] += 1
max_bots = max(counts.values())
print(f"max_bots: {max_bots}")
for point in [key for key, value in counts.items() if value == max_bots]:
    print(point, manhattan_distance((0,0,0), point))





68921
max_bots: 5
(12, 12, 12) 36


In [12]:
# interesting solution from u/EriiKKo

import sys,re
from queue import PriorityQueue

bots = [map(int, re.findall("-?\d+", line)) for line in input_lines]
q = PriorityQueue()
for x,y,z,r in bots:
    d = abs(x) + abs(y) + abs(z)
    q.put((max(0, d - r),1))
    q.put((d + r + 1,-1))
count = 0
maxCount = 0
result = 0
while not q.empty():
    dist,e = q.get()
    count += e
    if count > maxCount:
        result = dist
        maxCount = count
print(result)

36


In [9]:

nanobots

{(10, 12, 12): 2,
 (12, 14, 12): 2,
 (16, 12, 12): 4,
 (14, 14, 14): 6,
 (50, 50, 50): 200,
 (10, 10, 10): 5}

In [18]:
from queue import PriorityQueue

def closest_max_intersection(points):
    origin = (0, 0, 0)
    queue = PriorityQueue()

    for point, radius in points.items():
        distance = manhattan_distance(origin, point)
        # entry event
        queue.put((max(0, distance - radius), 1))
        # exit event
        queue.put((distance + radius + 1, -1))

    # init variables
    count = 0
    max_count = 0
    result = 0
    # process queue
    while not queue.empty():
        # get distance and event
        distance, event = queue.get()
        # increment/decrement count
        count += event
        # if new max
        if count > max_count:
            # update result and max
            result = distance
            max_count = count
    return result

closest_max_intersection(nanobots)

36